In [1]:
%load_ext autoreload
%autoreload 2

import torch
from timeit import default_timer
device = torch.device("cuda")
import torch.nn as nn
import numpy as np
import imageio
import matplotlib.pyplot as plt
from src.preprocessing import get_clear_data_idxs, min_max_scaller, load_mask

In [2]:
data_path = "./data/"
radar_data = np.load(f'{data_path}/radars_2020-11-01_2022-12-31.npy')
glorys     = np.load(f'{data_path}/glorys_2020-11-01_2022-12-31.npy')
gfs = np.load(f'{data_path}/gfs_2020-11-01_2022-12-31.npy')
mask = load_mask(data_path+'mask.png')


print('radar data : ', radar_data.shape)
print('glorys data :', glorys.shape)
print('gfs data :', gfs.shape)

radar data :  (791, 825, 200)
glorys data : (791, 3, 825, 200)
gfs data : (791, 3, 825, 200)


In [3]:
idxs = get_clear_data_idxs(radar_data, zero_th_percent = 80, mask = mask)
pairs = []
i = 0
while i < (len(idxs[0]) - 1):
    if (idxs[0][i] - idxs[0][i + 1]) == -1:
        pairs.append([idxs[0][i], idxs[0][i+1]])
        i += 2
    else:
        i += 1

In [4]:
train_pairs = np.array(pairs[:int(166 * 0.7)])
valid_pairs = np.array(pairs[int(166 * 0.7):int(166 * 0.9)])
test_pairs = np.array(pairs[int(166 * 0.9):])

In [5]:
batch_size = 50
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(radar_data[train_pairs[:, 0]]), 
                                                                          torch.Tensor(radar_data[train_pairs[:, 1]])), 
                                                                           batch_size=batch_size, shuffle=True, 
                                                                           pin_memory=True)


valid_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(radar_data[valid_pairs[:, 0]]), 
                                                                          torch.Tensor(radar_data[valid_pairs[:, 1]])), 
                                                                          batch_size=batch_size, shuffle=False, 
                                                                          pin_memory=True)


test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(torch.Tensor(radar_data[test_pairs[:, 0]]), 
                                                                         torch.Tensor(radar_data[test_pairs[:, 1]])), 
                                                                         batch_size=batch_size, shuffle=False, 
                                                                         pin_memory=True)


In [6]:
for x_train, y_train in train_loader:
    print(x_train.shape)
    break

torch.Size([50, 825, 200])


In [7]:
from radars.UNet import UNet

model = UNet(1, 1, False, 3)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)


In [10]:
def train(device, model, optimizer, trainloader, epochs=20, evalloader=False, name_experiment='model_test'):
    train_loss_list = []
    valid_loss_list = []
    grad_list = []
    
    for ep in range(epochs):
        percent_epoch  = ep/epochs
        model.train()
        t_1 = default_timer()
        train_pred_full = 0
        total_grad = 0
        for x_train, y_train in trainloader:
            x_train = x_train.unsqueeze(1)
            y_train = y_train.unsqueeze(1)
            
            l_recons = 0
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            
            bs = x_train.shape[0]
            pred = model(x_train)
            loss = model.loss(pred.reshape(bs, -1), y_train.reshape(bs, -1))

            train_pred_full += loss.item()

            optimizer.zero_grad()
            loss.backward()
            grad_by_batch  =  0
            for tag, value in model.named_parameters():
                if value.grad is not None:
                    grad =  value.grad.norm()
                    total_grad += grad 
                    grad_by_batch += grad

            optimizer.step()
            
        grad_list.append(total_grad.cpu().item())

        train_pred_full = train_pred_full / len(trainloader)

        train_loss_list.append(train_pred_full)

        t2 = default_timer()
        valid_pred_full = 0

        if evalloader:
            model.eval()
            with torch.no_grad():
                for x_valid, y_valid in evalloader:
                    loss = 0
                    x_valid = x_valid.to(device)
                    y_valid = y_valid.to(device)
                    bs = x_valid.shape[0]
                    
                    pred = model(x_valid)

                    l_pred = model.loss(pred.reshape(bs, -1), y_valid.reshape(bs, -1))

                    valid_pred_full += l_pred.item()

            valid_pred_full = valid_pred_full / len(evalloader)

            valid_loss_list.append(valid_pred_full)

        #scheduler.step(loss)
        # scheduler.step()

        if evalloader:
            if ep == 0:
                print("Epoch", "Time", "[Train Pred MSE]", "[Eval Pred MSE], [total grad], [grad_by_batch]")
            if ep % 50 == 0:
                print(ep, t2 - t_1, train_pred_full, valid_pred_full, total_grad, grad_by_batch)


        if ep % 99 == 0:
            torch.save(model.state_dict(), f"best_models_weights/{name_experiment}_{ep}.pth")
            
    return train_loss_list, valid_loss_list, grad_list

In [11]:
train_loss_list, valid_loss_list, grad_list = train(device, model, optimizer, 
                                         train_loader, epochs=1400, 
                                         evalloader=valid_loader)



: 

: 